<a href="https://colab.research.google.com/github/bu11ymaguire/studying/blob/main/ReinforcementLearning/GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GridWorld

In [5]:
import numpy as np

class GridWorld:
  def __init__(self):
    self.action_sapce = [0,1,2,3]
    self.action_meaning = {
        0: "UP",
        1: "DOWN",
        2: "LEFT",
        3: "RIGHT"
    }

    self.reward_map = np.array(
        [[0,0,0,1.0],
         [0,None,0,-1.0],
            [0,0,0,0]
        ]
    )

    self.goal_stat = (0,3) #목표 지점
    self.wall_state = (1,1) #벽
    self.current_state = (2,0) #시작 지점
    self.agent_state = self.start_state

    @property
    def height(self):
      return len(self.reward_map)

    @property
    def width(self):
      return len(self.reward_map[0])

    @property
    def shape(self):
      return self.reward_map.shape

    @property
    def actions(self):
      return self.action_space

In [ ]:
r